In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier
import pickle
import os
import warnings

#Load your data
train_data = pd.read_csv("../Data/train_data.csv")
test_data = pd.read_csv("../Data/test_data.csv")

#Separate features and labels
X_train = train_data.drop(columns=["GradeClass", "StudentID"])
y_train = train_data["GradeClass"].astype(int)

X_test = test_data.drop(columns=["GradeClass", "StudentID"])
y_test = test_data["GradeClass"].astype(int)

#Train XGBoost Model (no scaling needed)
model = XGBClassifier(n_estimators=100, eval_metric='mlogloss', random_state=42)
model.fit(X_train, y_train)

#Make predictions
y_pred = model.predict(X_test)

#Grade mapping
grade_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'F'}
y_test_letters = pd.Series(y_test).map(grade_map)
y_pred_letters = pd.Series(y_pred).map(grade_map)
label_names = [grade_map[i] for i in sorted(grade_map)]

#Evaluation
accuracy = accuracy_score(y_test_letters, y_pred_letters)
print("XGBoost Accuracy:", round(accuracy, 4), "\n────────────────────────")
print("\nClassification Report:")
print(classification_report(y_test_letters, y_pred_letters, target_names=label_names, zero_division=0))

#Confusion Matrix
ConfusionMatrixDisplay.from_estimator(
    model,
    X_test,
    y_test,
    display_labels=label_names,
    cmap='Blues'
)
plt.title(f"Confusion Matrix (Accuracy: {round(accuracy, 4)})")
plt.tight_layout()
plt.show()

#Save model only (no scaler needed)
output_model_path = "../Artifacts/PLK"
os.makedirs(output_model_path, exist_ok=True)

with open(os.path.join(output_model_path, "xgboost_model.pkl"), "wb") as f:
    pickle.dump(model, f)

#Prediction comparison table
comparison_df = test_data[["StudentID", "GradeClass"]].copy()
comparison_df["Predicted"] = y_pred
comparison_df["ActualGrade"] = comparison_df["GradeClass"].astype(int).map(grade_map)
comparison_df["PredictedGrade"] = pd.Series(y_pred).astype(int).map(grade_map)
comparison_df["Match"] = comparison_df["ActualGrade"] == comparison_df["PredictedGrade"]
comparison_df = comparison_df.sample(frac=1, random_state=42).reset_index(drop=True)

#Save prediction table
os.makedirs("project testing", exist_ok=True)
comparison_df.to_csv("../Artifacts/Predictions/prediction_results_xgboost.csv", index=False)

#Display sample
try:
    from IPython.display import display

    def highlight_false_text(row):
        styles = []
        for col in row.index:
            if col == "Match" and row["Match"] == False:
                styles.append("color: red; background-color: black")
            else:
                styles.append("background-color: black; color: white")
        return styles

    print("First 20 Predictions:")
    display(comparison_df.head(20).style.apply(highlight_false_text, axis=1))
except:
    print("\nFirst 20 Predictions:")
    print

# Grade mapping
grade_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'F'}
y_pred_letters = pd.Series(y_pred).map(grade_map)

# Count predicted grades
predicted_counts = y_pred_letters.value_counts().reindex(['A', 'B', 'C', 'D', 'F'], fill_value=0)

# Create summary DataFrame
grade_summary_df = pd.DataFrame({
    "Grade": predicted_counts.index,
    "Count": predicted_counts.values
})

# Add total row
total_row = pd.DataFrame([{"Grade": "Total", "Count": grade_summary_df["Count"].sum()}])
grade_summary_df = pd.concat([grade_summary_df, total_row], ignore_index=True)

# Display table
print(grade_summary_df.to_string(index=False))

# Plot bar chart
plt.figure(figsize=(8, 5))
plt.bar(grade_summary_df["Grade"], grade_summary_df["Count"], color='skyblue')
for i, count in enumerate(grade_summary_df["Count"]):
    plt.text(i, count + 2, str(count), ha='center', va='bottom', fontsize=10)
plt.title("Predicted Grade Distribution")
plt.xlabel("Grade")
plt.ylabel("Number of Students")
plt.tight_layout()
plt.show()